In [1]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [2]:
#ödev 3 teki veri
df = pd.read_excel('DataFrame.xlsx')

In [3]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,pp,bmi
0,50,2,168,62,110,80.0,1,1,0,0,1,0,30.0,21.967120
1,55,1,156,85,140,90.0,3,1,0,0,1,1,50.0,34.927679
2,51,1,165,64,130,70.0,3,1,0,0,0,1,60.0,23.507805
3,48,2,169,82,150,100.0,1,1,0,0,1,1,50.0,28.710479
4,47,1,156,56,100,60.0,1,1,0,0,0,0,40.0,23.011177


# Neden 3 farklı model olduğu, ödev 2'de anlatılmıştır.

In [4]:
# girdi (X) ve çıktı (y) verilerini ayırma
X = df.drop('cardio', axis=1)
y = df['cardio']

# veriyi eğitim ve test setlerine bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# veriyi ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modeli seçme ve eğitme
model = XGBClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Tahmin yapma
y_pred = model.predict(X_test_scaled)

# Modelin performansını değerlendirme
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.719296875
Confusion Matrix:
[[5101 1481]
 [2112 4106]]
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.77      0.74      6582
           1       0.73      0.66      0.70      6218

    accuracy                           0.72     12800
   macro avg       0.72      0.72      0.72     12800
weighted avg       0.72      0.72      0.72     12800



In [5]:
# karar eşiği için başlangıç değeri
best_threshold = 0

# en iyi recall (duyarlılık) değeri için başlangıç değeri
best_recall = 0

# modelin predict_proba fonksiyonunu kullanarak olasılık değerlerini alma
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# belirlenen eşik değerlerini deneyerek en iyi eşiği bulma
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    # eşik değerine göre tahminler
    y_pred_threshold = (y_pred_proba > threshold).astype(int)
    
    # her eşik değeri için Recall (duyarlılık) hesaplama
    recall = recall_score(y_test, y_pred_threshold)
    
    # en iyi recall değerini bulma
    if recall > best_recall:
        best_recall = recall
        best_threshold = threshold

print(f"En İyi Eşik Değeri: {best_threshold}")
print(f"En İyi Recall (Duyarlılık): {best_recall}")

# en iyi eşik değeri ile tahmin yapma
y_pred_best_threshold = (y_pred_proba > best_threshold).astype(int)

# Modelin performansını değerlendirme
accuracy_best_threshold = accuracy_score(y_test, y_pred_best_threshold)
conf_matrix_best_threshold = confusion_matrix(y_test, y_pred_best_threshold)
classification_rep_best_threshold = classification_report(y_test, y_pred_best_threshold)

print(f'En İyi Eşik Değeri ile Accuracy: {accuracy_best_threshold}')
print(f'En İyi Eşik Değeri ile Confusion Matrix:\n{conf_matrix_best_threshold}')
print(f'En İyi Eşik Değeri ile Classification Report:\n{classification_rep_best_threshold}')


En İyi Eşik Değeri: 0.1
En İyi Recall (Duyarlılık): 0.9880990672241878
En İyi Eşik Değeri ile Accuracy: 0.526875
En İyi Eşik Değeri ile Confusion Matrix:
[[ 600 5982]
 [  74 6144]]
En İyi Eşik Değeri ile Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.09      0.17      6582
           1       0.51      0.99      0.67      6218

    accuracy                           0.53     12800
   macro avg       0.70      0.54      0.42     12800
weighted avg       0.70      0.53      0.41     12800



In [10]:
# tahmin olasılıklarını al
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

#başlangıç değerleri
best_threshold = 0
best_metric_sum = 0

# 0.01 ile 1 arasında 0.01 arttırarak eşik değerlerini deneme
for threshold in np.arange(0.01, 1, 0.01):
    # Eşik değerine göre tahminleri al
    y_pred_threshold = (y_pred_proba > threshold).astype(int)
    
    # metrikleri hesaplama
    accuracy = accuracy_score(y_test, y_pred_threshold)
    precision = precision_score(y_test, y_pred_threshold)
    recall = recall_score(y_test, y_pred_threshold)
    f1 = f1_score(y_test, y_pred_threshold)
    
    # metric sum hesaplama
    metric_sum = accuracy + precision + recall + f1

    # en yüksek metric sum ı bulma
    if metric_sum > best_metric_sum:
        best_metric_sum = metric_sum
        best_threshold = threshold

print(f"En İyi Eşik Değeri: {best_threshold}")

# En iyi eşik değeri ile tahmin yapma
y_pred_best_threshold = (y_pred_proba > best_threshold).astype(int)

# Modelin performansını değerlendirme
accuracy_best_threshold = accuracy_score(y_test, y_pred_best_threshold)
conf_matrix_best_threshold = confusion_matrix(y_test, y_pred_best_threshold)
classification_rep_best_threshold = classification_report(y_test, y_pred_best_threshold)

print(f'En İyi Eşik Değeri ile Accuracy: {accuracy_best_threshold}')
print(f'En İyi Eşik Değeri ile Confusion Matrix:\n{conf_matrix_best_threshold}')
print(f'En İyi Eşik Değeri ile Classification Report:\n{classification_rep_best_threshold}')

En İyi Eşik Değeri: 0.35000000000000003
En İyi Eşik Değeri ile Accuracy: 0.691796875
En İyi Eşik Değeri ile Confusion Matrix:
[[3857 2725]
 [1220 4998]]
En İyi Eşik Değeri ile Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.59      0.66      6582
           1       0.65      0.80      0.72      6218

    accuracy                           0.69     12800
   macro avg       0.70      0.69      0.69     12800
weighted avg       0.71      0.69      0.69     12800

